.. meta::
   :description: A guide which introduces the most important steps to get started with pymoo, an open-source multi-objective optimization framework in Python.

.. meta::
   :keywords: Multi-objective Optimization, Python, Evolutionary Computation, Optimization Test Problem, Hypervolume

In [2]:
import csv
import numpy as np

from pymoo.factory import get_problem
from pymoo.optimize import minimize
problem_list = ["zdt1","zdt2","zdt3","zdt4","zdt5","zdt6",
                "wfg1","wfg2","wfg3","wfg4","wfg5","wfg6","wfg7","wfg8","wfg9"]
n_vars = [None, None, None, None, None, None, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
n_objs = [None, None, None, None, None, None, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
n_pareto_points = 1000

from pymoo.performance_indicator.igd import IGD
from pymoo.performance_indicator.hv import Hypervolume
hv_ref_point = np.array([1.0, 1.0])

#test function
def test_alg(algorithm, n_runs, report_file_name):
    with open(report_file_name, mode='w+') as result_file:
        writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["problem_name", "n_var", "n_obj", "n_eval", 
                         "mean_rt", "std_rt",  
                         "mean_igd", "std_igd", "min_igd", "max_igd", 
                         "mean_hv", "std_hv", "min_hv", "max_hv"])

        for idx in range(len(problem_list)):
            problem_name = problem_list[idx]
            n_var = n_vars[idx]
            n_obj = n_objs[idx]

            if n_var is not None:
                problem = get_problem(problem_name, n_var=n_var, n_obj=n_obj)
            else:
                problem = get_problem(problem_name)

            igd_vals = np.array([])
            hv_vals = np.array([])
            exec_times = np.array([])

            try:
                pf = problem.pareto_front(n_pareto_points=n_pareto_points)
            except:
                pf = problem.pareto_front(n_points=n_pareto_points)

            for r in range(n_runs):
                res = minimize(problem, algorithm, ("n_eval", 25000), verbose=False)
                igd_val = IGD(pf=pf, normalize=False).calc(res.F)
                igd_vals = np.append(igd_vals, igd_val)

                hv_val = Hypervolume(ref_point=hv_ref_point, normalize=True, pf=pf).calc(res.F)
                hv_vals = np.append(hv_vals, hv_val)

                exec_times = np.append(exec_times, res.exec_time)
        
            writer.writerow([problem_name, problem.n_var, problem.n_obj, 25000, 
                         np.mean(exec_times), np.std(exec_times),  
                         np.mean(igd_vals), np.std(igd_vals), np.min(igd_vals), np.max(igd_vals),
                         np.mean(hv_vals), np.std(hv_vals), np.min(hv_vals), np.max(hv_vals)] )     

In [2]:
# #test nsga2
# from pymoo.algorithms.nsga2 import NSGA2
# from pymoo.operators.crossover.simulated_binary_crossover import SimulatedBinaryCrossover
# from pymoo.operators.mutation.polynomial_mutation import PolynomialMutation

# sbx = SimulatedBinaryCrossover(eta=20, prob=0.9)
# pm = PolynomialMutation(prob=None, eta=20)
# algorithm = NSGA2(crossover=sbx, mutation=pm)
     
# test_alg(algorithm, 1, "nsga2.csv")

In [3]:
#test moead_alfpa
from pymoo.algorithms.moead_fpa import MOEAD_ALFPA
from pymoo.factory import get_problem, get_visualization, get_reference_directions
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.decomposition.tchebicheff import Tchebicheff

n_obj = 2
ref_dirs = get_reference_directions("das-dennis", n_obj, n_partitions=99)
algorithm = MOEAD_ALFPA(ref_dirs, decomposition=Tchebicheff())

test_alg(algorithm, 20, "moead_alfpa_357.csv")

/home/gemilang/python/python-mop/src/mypymoo/pymoo/algorithms/moead_fpa.py:199: RuntimeWarning: invalid value encountered in true_divide
  G = (FV - off_FV)/FV
/home/gemilang/python/python-mop/src/mypymoo/pymoo/algorithms/moead_fpa.py:199: RuntimeWarning: divide by zero encountered in true_divide
  G = (FV - off_FV)/FV
/home/gemilang/python/python-mop/src/mypymoo/pymoo/operators/mutation/polynomial_mutation.py:47: RuntimeWarning: invalid value encountered in power
  d = np.power(val, mut_pow) - 1.0
/home/gemilang/python/python-mop/src/mypymoo/pymoo/algorithms/moead_fpa.py:147: RuntimeWarning: invalid value encountered in true_divide
  FRR = np.dot(current_mask, self.op_rewards)/tot_rewards
/home/gemilang/python/python-mop/src/mypymoo/pymoo/operators/mutation/polynomial_mutation.py:52: RuntimeWarning: invalid value encountered in power
  d = 1.0 - (np.power(val, mut_pow))
/home/gemilang/python/python-mop/src/mypymoo/pymoo/decomposition/tchebicheff.py:27: RuntimeWarning: invalid value en

KeyboardInterrupt: 

In [4]:
#test mofpa

In [2]:
#test moead_cs
from pymoo.algorithms.moead_cs import MOEAD_CS
from pymoo.factory import get_problem, get_visualization, get_reference_directions
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

ref_dirs = get_reference_directions("das-dennis", 2, n_partitions=99)
algorithm = MOEAD_CS(ref_dirs=ref_dirs)

test_alg(algorithm, 1, "moead_cs.csv")

/home/gemilang/python/python-mop/src/mypymoo/pymoo/algorithms/moead_cs.py:143: RuntimeWarning: invalid value encountered in true_divide
  cos = np.dot(f, w.T) / np.dot(np.linalg.norm(f, axis=1), np.linalg.norm(w, axis=1))
/home/gemilang/python/python-mop/src/mypymoo/pymoo/algorithms/moead_cs.py:200: RuntimeWarning: divide by zero encountered in true_divide
  G = (FV - off_FV)/FV


In [ ]:
A = np.array([1,2,3,4,5])
B = np.array([2,2,4,5,6])
worse = np.any(A > B)
exist_better = np.any(A < B)
print(not worse and exist_better)